# Google combined tools example

This notebook features a more advanced usage of Agent Tools, using the Google Calendar, Mail and Search integrations as well as the Load and Search Meta Tool to fufill a more complicated set of tasks for the user.

## Setup the Tools
First we will import OpenAI and setup the Agent:

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-api-key"

from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

Now we can import the Google Tools we are going to use. See the README for the respective tools to get started with authentication.

In [ ]:
from llama_index.tools.gmail.base import GmailToolSpec
from llama_index.tools.google_calendar.base import GoogleCalendarToolSpec
from llama_index.tools.google_search.base import GoogleSearchToolSpec

gmail_tools = GmailToolSpec().to_tool_list()
gcal_tools = GoogleCalendarToolSpec().to_tool_list()
gsearch_tools = GoogleSearchToolSpec(key="api-key", engine="engine").to_tool_list()

Let's take a look at all of the tools we have available from the 3 tool specs we initialized:

In [ ]:
for tool in [*gmail_tools, *gcal_tools, *gsearch_tools]:
    print(tool.metadata.name)
    print(tool.metadata.description)

load_data
load_data() -> List[llama_index.schema.Document]
Load emails from the user's account
search_messages
search_messages()

create_draft
create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str
Create and insert a draft email.
           Print the returned draft's message and id.
           Returns: Draft object, including draft id and message meta data.

        Args:
            to (Optional[str]): The email addresses to send the message to
            subject (Optional[str]): The subject for the event
            message (Optional[str]): The message for the event
        
update_draft
update_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None, draft_id: str = None) -> str
Update a draft email.
           Print the returned draft's message and id.
           This function is required to be passed a draft_id that is obtained when creating messages
           Returns: Draft obj

We have to be conscious of the models context length when using these tools as if we are not careful the response can easily be larger than the token limit. In particular, the load_data function for emails returns large payloads, as well as google search. In this example I will wrap those two tools in the Load and Search Meta tool:

In [ ]:
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

print("Wrapping " + gsearch_tools[0].metadata.name)
gsearch_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
    gsearch_tools[0],
).to_tool_list()

print("Wrapping gmail " + gmail_tools[0].metadata.name)
gmail_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
    gmail_tools[0],
).to_tool_list()

print("Wrapping google calendar " + gcal_tools[0].metadata.name)
gcal_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
    gcal_tools[0],
).to_tool_list()

Notice we are only wrapping individual tools out of the tool list. Lets combine the all the tools together into a combined list:

In [ ]:
all_tools = [
    *gsearch_load_and_search_tools,
    *gmail_load_and_search_tools,
    *gcal_load_and_search_tools,
    *gcal_tools[1::],
    *gmail_tools[1::],
    *gsearch_tools[1::],
]

Now the tools are ready to pass to the agent:

In [ ]:
agent = FunctionAgent(tools=all_tools, llm=OpenAI(model="gpt-4.1"))

## Interacting with the Agent

We are now ready to interact with the Agent and test the calendar, email and search capabilities! Let's try out the search first:

In [ ]:
await agent.run(
    "search google and find the email address for a dentist in toronto near bloor and"
    " dufferin"
)

## Summary

We were able to use the google search, email and calendar tools to find a dentist and request an appointment, making sure any calendar conflicts were avoided. This notebook should prove useful for experimenting with more complicated agents, combining together the tools available in LlamaHub to create more complicated workflows the agent can execute.

